In [59]:
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import MaxPooling3D, Conv2D, MaxPooling2D, SeparableConv2D, ConvLSTM2D, TimeDistributed, UpSampling2D, Concatenate
from keras.layers import Activation, BatchNormalization
from keras.optimizers import SGD, Adadelta, Adam
from keras import Input
from keras import Model

In [69]:
def strange_network(input_shape):
    input_img =  Input(input_shape)
    c_l1 = ConvLSTM2D(filters=32,kernel_size=(2,2), padding = 'same', return_sequences=True) (input_img)
    c_l2 = ConvLSTM2D(filters=32,kernel_size=(2,2), padding = 'same', return_sequences=True) (c_l1)
    c_l3 = ConvLSTM2D(filters=32,kernel_size=(2,2), padding = 'same', return_sequences=True) (c_l2)
    pool_1 = TimeDistributed(MaxPooling2D((3,3), padding='same')) (c_l3)
    c_l4 = ConvLSTM2D(filters=32,kernel_size=(2,2), padding = 'same', return_sequences=True) (pool_1)
    c_l5 = ConvLSTM2D(filters=32,kernel_size=(2,2), padding = 'same', return_sequences=True) (c_l4)
    c_l6 = ConvLSTM2D(filters=32,kernel_size=(2,2), padding = 'same', return_sequences=True) (c_l5)
    
    upsample_1 = TimeDistributed(UpSampling2D((3,3))) (c_l6)
    conv_1 = TimeDistributed(Conv2D(32, (3,3)))(upsample_1)
    merge = Concatenate()([c_l3, conv_1])
    
    conv_lc = ConvLSTM2D(filters=16,kernel_size=(1,1), padding = 'same', return_sequences=False) (merge)
    conv_out_cc = ConvLSTM2D(filters=1,kernel_size=(1,1), padding = 'same', return_sequences=False)(merge)

    model = Model(inputs=input_img, outputs=[conv_lc, conv_out_cc])
    return model

    


In [70]:
model = strange_network((4, 64, 64, 7))

In [71]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_23 (InputLayer)           (None, 4, 64, 64, 7) 0                                            
__________________________________________________________________________________________________
conv_lst_m2d_105 (ConvLSTM2D)   (None, 4, 64, 64, 32 20096       input_23[0][0]                   
__________________________________________________________________________________________________
conv_lst_m2d_106 (ConvLSTM2D)   (None, 4, 64, 64, 32 32896       conv_lst_m2d_105[0][0]           
__________________________________________________________________________________________________
conv_lst_m2d_107 (ConvLSTM2D)   (None, 4, 64, 64, 32 32896       conv_lst_m2d_106[0][0]           
__________________________________________________________________________________________________
time_distr